In [1]:
!pip install gurobipy

     ---------------------------------------- 9.7/9.7 MB 322.8 kB/s eta 0:00:00


In [3]:
import gurobipy as gp
from gurobipy import GRB

In [4]:
#parameters

budget = 20
regions, population = gp.multidict({
    0: 523, 1: 690, 2: 420,
    3: 1010, 4: 1200, 5: 850,
    6: 400, 7: 1008, 8: 950
})

In [5]:
m = gp.Model("Cell_Tower")

build = m.addVars(len(sites), vtype=GRB.BINARY, name="Build")
is_covered = m.addVars(len(regions), vtype=GRB.BINARY, name="Is_covered")

m.addConstrs((gp.quicksum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")
m.addConstr(build.prod(cost) <= budget, name="budget")

m.setObjective(is_covered.prod(population), GRB.MAXIMIZE)

m.optimize()

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-1005G1 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 10 rows, 15 columns and 36 nonzeros
Model fingerprint: 0xf0a21eec
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 21 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Found heuristic solution: objective 7051.0000000

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 7051 -0 


In [6]:
# display optimal values of decision variables

for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        print(f"\n Build a cell tower at location Tower {tower}.")


 Build a cell tower at location Tower 0.

 Build a cell tower at location Tower 2.

 Build a cell tower at location Tower 4.


In [7]:
# Percentage of the population covered by the cell towers built is computed as follows.

total_population = 0

for region in range(len(regions)):
    total_population += population[region]

coverage = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage associated to the cell towers build plan is: {coverage} %")


 The population coverage associated to the cell towers build plan is: 100.0 %


In [8]:
# Percentage of budget consumed to build cell towers

total_cost = 0

for tower in range(len(sites)):
    if (abs(build[tower].x) > 0.5):
        total_cost += cost[tower]*int(build[tower].x)

budget_consumption = round(100*total_cost/budget, 2)

print(f"\n The percentage of budget consumed associated to the cell towers build plan is: {budget_consumption} %")


 The percentage of budget consumed associated to the cell towers build plan is: 71.0 %
